In [1]:
import os
import scipy.io as sio
import joblib
import schema
import traceback
import sklearn
import path
import pandas as pd
import numpy as np
import collections
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyspark


In [2]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types as sp_types
from pyspark.sql import functions as F
from sklearn import preprocessing

spark = SparkSession.builder.master("local").appName("msg10").getOrCreate()

In [3]:
%matplotlib inline

In [4]:
import tensorflow as tf
tf.__version__

D:\environment\python\anaconda352\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.12.0'

数据说明：
1. 对于PSG数据
   - stage 睡眠所处阶段
   - 3 -> 醒来， 2 -> 浅睡， 1 -> 深睡

In [5]:
data_root=r"E:\datasources\MS10Data"
p = r"E:\datasources\MS10Data\1\psg\psg.mat"
rdp=r"E:\datasources\MS10Data\1\raw\rawMat.mat"
save_root=r"E:\datasources\MS10DataTF"

In [6]:
psg_data_paths=[os.path.join(data_root,f"{i}\\psg\\psg.mat") for i in range(1,33)]
raw_data_paths=[os.path.join(data_root,f"{i}\\raw\\rawMat.mat") for i in range(1,33)]
psg_raw_data_paths=list(zip(psg_data_paths,raw_data_paths))


In [7]:
def parse_mat_psg(fp):
    psg1=sio.loadmat(fp)
    ti=np.apply_along_axis(lambda row:row[0],0,psg1["psgStage"]['Time']).flatten()
    st=np.apply_along_axis(lambda row:row[0],0,psg1["psgStage"]['stage']).flatten()
    st=np.apply_along_axis(lambda x:int(x),1,st.reshape(-1,1))
    print(st.shape)
    result_df = pd.DataFrame(np.vstack([ti,st]).transpose(),columns=['time','stage'])
    result_df.loc[:,'idx']=result_df.index
    return result_df

def parse_mat_signal(fp):
    rs=sio.loadmat(fp)
    r1=rs['rawData']['dataSignalOld'][0,0]
    r2=rs['rawData']['dataSignal'][0,0]
    r3=rs['rawData']['dataSwitch'][0,0]
    r4=rs['rawData']['Time'][0,0]
    result_df = pd.DataFrame(np.hstack([r1,r2,r3,r4]),columns=['data_signal_old','data_signal','data_switch','time'])
    result_df.loc[:,'idx']=result_df.index
    return result_df

def parse_mat_time(matlab_datenum):
    return datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)


def data_to_parquet(file_path,save_root,flag='signal'):
    status = False
    flag=schema.Or('signal','psg').validate(flag)
    save_file_path=os.path.join(save_root,"ms10_{0}_{1:>03s}.parquet".format(flag,path.Path(file_path).splitall()[-3]))
    print(save_file_path)
    try:
        if flag=='signal':
            parse_mat_signal(file_path).to_parquet(save_file_path, compression='gzip')
        else:
            parse_mat_psg(file_path).to_parquet(save_file_path, compression='gzip')
        status=True
    except Exception as e:
        traceback.print_exc()
    return status

mp=lambda it:(data_to_parquet(it[0],save_root=save_root,flag='psg'),data_to_parquet(it[1],save_root=save_root,flag='signal'))

In [8]:
# with joblib.Parallel(n_jobs=10, prefer="threads") as parallel:
#     parallel(joblib.delayed(mp)(it) for it in psg_raw_data_paths)

In [9]:
# 数据解析生成tf-record
# 定义特征对象
def _int64_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _make_named_features(named_features):
    """生成一个命名的特征，给特征对象赋予一个schema"""
    return tf.train.Features(feature=named_features)

# 定义序列record，多了一层包裹的抽象
def _feature_list(feature_list):
    """生成一个特征对象的序列"""
    return tf.train.FeatureList(feature=feature_list)

def _make_named_featurelists(named_featurelists):
    """生成一个命名的特征，给特征对象的序列赋予一个schema"""
    return tf.train.FeatureLists(feature_list=named_featurelists)

In [10]:
psg_sdf=spark.read.parquet(r"E:\datasources\MS10DataTF\ms10_psg_014.parquet")
psg_sdf=psg_sdf.drop(F.col("__index_level_0__"))
psg_sdf.show()

+-----------------+-----+---+
|             time|stage|idx|
+-----------------+-----+---+
|737236.9168981481|  3.0|  0|
|737236.9172453685|  3.0|  1|
|737236.9175925888|  3.0|  2|
|737236.9179398092|  3.0|  3|
|737236.9182870295|  3.0|  4|
|737236.9186342498|  3.0|  5|
|737236.9189814702|  3.0|  6|
|737236.9193286905|  3.0|  7|
|737236.9196759108|  3.0|  8|
|737236.9200231312|  3.0|  9|
|737236.9203703515|  3.0| 10|
|737236.9207175719|  3.0| 11|
|737236.9210647922|  3.0| 12|
|737236.9214120125|  3.0| 13|
|737236.9217592329|  3.0| 14|
|737236.9221064532|  3.0| 15|
|737236.9224536736|  3.0| 16|
|737236.9228008939|  3.0| 17|
|737236.9231481142|  3.0| 18|
|737236.9234953346|  3.0| 19|
+-----------------+-----+---+
only showing top 20 rows



In [11]:
signal_sdf=spark.read.parquet(r"E:\datasources\MS10DataTF\ms10_signal_014.parquet")
signal_sdf=signal_sdf.drop(F.col("__index_level_0__"))
signal_sdf.show()

+---------------+-----------+-----------+-----------------+---+
|data_signal_old|data_signal|data_switch|             time|idx|
+---------------+-----------+-----------+-----------------+---+
|        10884.0|     8416.8|        1.0|737236.7675810185|  0|
|        11072.0|     8454.4|        1.0|737236.7675810637|  1|
|        11196.0|     8479.2|        1.0| 737236.767581109|  2|
|        11260.0|     8492.0|        1.0|737236.7675811541|  3|
|        11328.0|     8505.6|        1.0|737236.7675811993|  4|
|        11336.0|     8507.2|        1.0|737236.7675812446|  5|
|        11364.0|     8512.8|        1.0|737236.7675812898|  6|
|        11392.0|     8518.4|        1.0| 737236.767581335|  7|
|        11356.0|     8511.2|        1.0|737236.7675813802|  8|
|        11352.0|     8510.4|        1.0|737236.7675814254|  9|
|        11344.0|     8508.8|        1.0|737236.7675814707| 10|
|        11340.0|     8508.0|        1.0|737236.7675815158| 11|
|        11356.0|     8511.2|        1.0

In [12]:
#from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import types as sp_types
from sklearn import preprocessing
scale_f=F.pandas_udf(lambda s:pd.Series(preprocessing.scale(s)),returnType=sp_types.FloatType(),functionType=F.PandasUDFType.SCALAR)

scaler= preprocessing.StandardScaler()

@F.pandas_udf(returnType=sp_types.FloatType(),functionType=F.PandasUDFType.SCALAR)
def scale_f2(s):
    rs=scaler.fit_transform(s.values.reshape(-1,1))
    print(f"s.size-> {s.size}")
    return pd.Series(rs.flatten())
    

two_fold_f=F.udf(lambda value: value*2,returnType=sp_types.FloatType())

In [13]:
pdf=pd.read_parquet(r"E:\datasources\MS10DataTF\ms10_signal_014.parquet")
pdf.head()

,data_signal_old,data_signal,data_switch,time,idx
0,10884.0,8416.8,1.0,737236.767581,0
1,11072.0,8454.4,1.0,737236.767581,1
2,11196.0,8479.2,1.0,737236.767581,2
3,11260.0,8492.0,1.0,737236.767581,3
4,11328.0,8505.6,1.0,737236.767581,4


In [14]:
scaler=preprocessing.StandardScaler()
scaler.fit_transform(pdf['data_signal'].values.reshape(-1,1))

array([[ 0.8865163 ],
       [ 0.94245236],
       [ 0.97934635],
       ...,
       [-0.0132211 ],
       [-0.01084084],
       [-0.0132211 ]])

In [15]:
scaler.mean_,scaler.scale_

(array([7820.88717012]), array([672.19613375]))

In [16]:
dc3=signal_sdf.withColumn("scale_data_signal601",scale_f2(F.col('data_signal')))
dc3.show()

+---------------+-----------+-----------+-----------------+---+--------------------+
|data_signal_old|data_signal|data_switch|             time|idx|scale_data_signal601|
+---------------+-----------+-----------+-----------------+---+--------------------+
|        10884.0|     8416.8|        1.0|737236.7675810185|  0|           2.1085901|
|        11072.0|     8454.4|        1.0|737236.7675810637|  1|            2.244437|
|        11196.0|     8479.2|        1.0| 737236.767581109|  2|           2.3340383|
|        11260.0|     8492.0|        1.0|737236.7675811541|  3|           2.3802838|
|        11328.0|     8505.6|        1.0|737236.7675811993|  4|             2.42942|
|        11336.0|     8507.2|        1.0|737236.7675812446|  5|           2.4352007|
|        11364.0|     8512.8|        1.0|737236.7675812898|  6|           2.4554331|
|        11392.0|     8518.4|        1.0| 737236.767581335|  7|           2.4756658|
|        11356.0|     8511.2|        1.0|737236.7675813802|  8|  

In [17]:
dc3.count()

14817280

In [18]:
ps=pd.Series(np.arange(10))
rs2=ps.values.reshape(-1,1)

In [19]:
m1=signal_sdf.agg(F.mean(F.col('data_signal'))).first().asDict()['avg(data_signal)']
std1=signal_sdf.agg(F.stddev(F.col('data_signal'))).first().asDict()['stddev_samp(data_signal)']

In [20]:
ps.size

10

In [21]:
dc4=signal_sdf.withColumn("scale_data_signal61",(F.col('data_signal')-m1)/std1)

In [22]:
dc4.show()

+---------------+-----------+-----------+-----------------+---+-------------------+
|data_signal_old|data_signal|data_switch|             time|idx|scale_data_signal61|
+---------------+-----------+-----------+-----------------+---+-------------------+
|        10884.0|     8416.8|        1.0|737236.7675810185|  0| 0.8865162708447009|
|        11072.0|     8454.4|        1.0|737236.7675810637|  1| 0.9424523240939083|
|        11196.0|     8479.2|        1.0| 737236.767581109|  2| 0.9793463166625358|
|        11260.0|     8492.0|        1.0|737236.7675811541|  3| 0.9983883773431158|
|        11328.0|     8505.6|        1.0|737236.7675811993|  4| 1.0186205668162336|
|        11336.0|     8507.2|        1.0|737236.7675812446|  5|  1.021000824401307|
|        11364.0|     8512.8|        1.0|737236.7675812898|  6| 1.0293317259490589|
|        11392.0|     8518.4|        1.0| 737236.767581335|  7| 1.0376626274968137|
|        11356.0|     8511.2|        1.0|737236.7675813802|  8| 1.0269514683

In [23]:
print(dc4.agg(F.mean(F.col('scale_data_signal61'))).first())
print(dc4.agg(F.stddev(F.col('scale_data_signal61'))).first())

Row(avg(scale_data_signal61)=-5.308867562187707e-12)
Row(stddev_samp(scale_data_signal61)=0.999999999999934)


In [24]:
pd.Series(np.arange(10))
pd.Series(preprocessing.scale(pd.Series(np.arange(10))))

D:\environment\python\anaconda352\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


0   -1.566699
1   -1.218544
2   -0.870388
3   -0.522233
4   -0.174078
5    0.174078
6    0.522233
7    0.870388
8    1.218544
9    1.566699
dtype: float64

In [25]:
sagg=signal_sdf.agg(F.mean(F.col('data_signal')))

In [26]:
sagg.first()


Row(avg(data_signal)=7820.887170125228)

In [27]:
dc=signal_sdf.withColumn("scale_data_signal",scale_f(F.col('data_signal')))
dcg=dc.groupBy(F.col('data_switch'))

In [28]:
dcg.agg()

AssertionError: exprs should not be empty

In [ ]:
signal_sdf.columns

In [ ]:
pandas_udf("integer", PandasUDFType.SCALAR)  # doctest: +SKIP

In [ ]:
#signal_sdf.filter(signal_sdf['time']==psg_sdf['time']).show()
signal_sdf.filter(signal_sdf.time.between(737236.7675813802,737236.7675814707)).show()

In [ ]:
from pyspark.sql import Row
df = spark.createDataFrame([Row(r=Row(a=1, b="b"))])

In [ ]:
r1=Row(a=1, b="b")
df.r.getField('a')

In [ ]:
df = spark.createDataFrame([([1, 2], {"key": "value"})], ["l", "d"])
df.show()

In [ ]:
left.show()

In [29]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=['data_signal'], outputCol="features")
stream_df = vecAssembler.transform(signal_sdf)
stream_df.show()

+---------------+-----------+-----------+-----------------+---+--------+
|data_signal_old|data_signal|data_switch|             time|idx|features|
+---------------+-----------+-----------+-----------------+---+--------+
|        10884.0|     8416.8|        1.0|737236.7675810185|  0|[8416.8]|
|        11072.0|     8454.4|        1.0|737236.7675810637|  1|[8454.4]|
|        11196.0|     8479.2|        1.0| 737236.767581109|  2|[8479.2]|
|        11260.0|     8492.0|        1.0|737236.7675811541|  3|[8492.0]|
|        11328.0|     8505.6|        1.0|737236.7675811993|  4|[8505.6]|
|        11336.0|     8507.2|        1.0|737236.7675812446|  5|[8507.2]|
|        11364.0|     8512.8|        1.0|737236.7675812898|  6|[8512.8]|
|        11392.0|     8518.4|        1.0| 737236.767581335|  7|[8518.4]|
|        11356.0|     8511.2|        1.0|737236.7675813802|  8|[8511.2]|
|        11352.0|     8510.4|        1.0|737236.7675814254|  9|[8510.4]|
|        11344.0|     8508.8|        1.0|737236.767

In [30]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",withStd=True, withMean=False)

In [31]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(stream_df)

In [32]:
scalerModel.mean,scalerModel.std

(DenseVector([7820.8872]), DenseVector([672.1962]))

In [33]:
# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(stream_df)
scaledData.show()

+---------------+-----------+-----------+-----------------+---+--------+--------------------+
|data_signal_old|data_signal|data_switch|             time|idx|features|      scaledFeatures|
+---------------+-----------+-----------+-----------------+---+--------+--------------------+
|        10884.0|     8416.8|        1.0|737236.7675810185|  0|[8416.8]|[12.521345026274926]|
|        11072.0|     8454.4|        1.0|737236.7675810637|  1|[8454.4]|[12.577281079524134]|
|        11196.0|     8479.2|        1.0| 737236.767581109|  2|[8479.2]|[12.614175072092763]|
|        11260.0|     8492.0|        1.0|737236.7675811541|  3|[8492.0]|[12.633217132773343]|
|        11328.0|     8505.6|        1.0|737236.7675811993|  4|[8505.6]|[12.653449322246463]|
|        11336.0|     8507.2|        1.0|737236.7675812446|  5|[8507.2]|[12.655829579831535]|
|        11364.0|     8512.8|        1.0|737236.7675812898|  6|[8512.8]|[12.664160481379287]|
|        11392.0|     8518.4|        1.0| 737236.767581335| 

In [34]:
scale_df=scaledData.rdd.map(lambda x:[round(float(y),3) for y in x['scaledFeatures']]).toDF(["signal"])

In [35]:
scale_df.show()
scale_df.cache()

+------+
|signal|
+------+
|12.521|
|12.577|
|12.614|
|12.633|
|12.653|
|12.656|
|12.664|
|12.672|
|12.662|
|12.661|
|12.658|
|12.657|
|12.662|
|12.662|
|12.678|
|12.699|
|12.717|
|12.743|
|12.767|
|12.803|
+------+
only showing top 20 rows



DataFrame[signal: double]

In [ ]:
scale_df.agg(F.avg('signal')).show()
#spark.createDataFrame(rdd,("signal"))

In [ ]:
fv=scaledData.first().scaledFeatures
fv.toArray()
pd.Series(np.arange(10))
from pyspark.ml.linalg import Vectors
psv=pd.Series([Vectors.dense([1.0,0.2]),Vectors.dense([2.0,12]),Vectors.dense([3.0,6])])

In [ ]:
av=psv[0].toArray()
psv.tolist()

In [ ]:
pd.DataFrame(psv.map(lambda x:x.toArray()).tolist())

In [ ]:
from pyspark.sql import Row
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
eDF.show()
eDF.select(F.posexplode(eDF.intlist)).show()

In [ ]:
from pyspark.sql import Row
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
eDF.show()
eDF.select(F.explode(eDF.intlist).alias("anInt")).show()

In [ ]:
plt.figure(figsize=(100,50))
plt.plot(signal['data_signal'],)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)


In [ ]:
signal=parse_mat_signal(rdp)
psg=parse_mat_psg(p)

psg_time=psg['time'].values
psg_stage=psg['stage'].values
psg_int=list(zip(psg_time[:-1],psg_time[1:],psg_stage[1:]))

def make_signal_psg_list(psg,signal):
    signal_list=[]
    psg_list=[]
    psg_time=psg['time'].values
    psg_stage=psg['stage'].values
    psg_int=list(zip(psg_time[:-1],psg_time[1:],psg_stage[1:]))
    for it in psg_int:
        signal_list.append(signal.query(f"(time>{it[0]})and(time<{it[1]})")['data_signal'].values)
        psg_list.append(it[2])
    return signal_list,psg_list

In [ ]:
signal_list,psg_list=make_signal_psg_list(psg,signal)

In [ ]:
c=0

named_featurelists=collections.defaultdict(list)

for sig,label in zip(signal_list,psg_list):
    c+=1
    if c<10:
        named_featurelists['signal_seq'].append(_bytes_feature(sig.tostring()))
        named_featurelists['psg_seq'].append(_int64_feature(int(label)))
        print(sig,label)
    res=_make_named_featurelists({k:_feature_list(v) for k,v in named_featurelists.items()})
    

In [ ]:
example = tf.train.SequenceExample(feature_lists=res)
filename='movie_ratings4.tfrecord'
if tf.gfile.Exists(filename):
    print(tf.gfile.Exists(filename))
    tf.gfile.Remove(filename)
    print(tf.gfile.Exists(filename))

with tf.python_io.TFRecordWriter(filename) as writer:
    writer.write(example.SerializeToString())
    writer.write(example.SerializeToString())
    writer.close()

In [ ]:
import pyspark

from pyspark.sql import SparkSession

In [ ]:
logFile = "tmp.txt"  # Should be some file on your system
spark = SparkSession.builder.master("local").appName("SimpleApp").getOrCreate()
logData = spark.read.text(logFile).cache()

numAs = logData.filter(logData.value.contains('a')).count()
numBs = logData.filter(logData.value.contains('b')).count()

print("Lines with a: %i, lines with b: %i" % (numAs, numBs))

#spark.stop()

In [ ]:
col=logData['value']
col.

In [ ]:
sequence_features = {
    'signal_seq': tf.FixedLenSequenceFeature([], dtype=tf.string),
    'psg_seq': tf.FixedLenSequenceFeature([], dtype=tf.int64),
}

dataset = tf.data.TFRecordDataset(filename)
#dataset = dataset.batch(1)
#dataset=dataset.shuffle(200)
dataset=dataset.repeat(4)
iterator=dataset.make_one_shot_iterator()
it=iterator.get_next()
ps=tf.parse_single_sequence_example(it,sequence_features=sequence_features)
ps_signal=tf.decode_raw(ps[1]['signal_seq'],tf.float64)

In [ ]:
ps_signal2=ps_signal[:,0:10]

In [ ]:
tfa=tf.convert_to_tensor(np.arange(144).reshape(-1,12))

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(tfs, maxlen=10, dtype='float32', padding='pre', truncating='pre', value=0.0)

In [ ]:
with tf.Session() as sess:
    print(sess.run(ps_signal).shape)
    print("-"*30)
    print(sess.run(ps_signal2))

In [ ]:
from sklearn import preprocessing

In [ ]:
preprocessing.scale(np.arange(144).reshape(-1,12))

In [ ]:
with tf.Session() as sess:
    #print(sess.run(np.dot(tfa,tfa)))
    print(sess.run(tf.reduce_sum(tfa)))
    #print(sess.run(np.mean(tfa)))

In [ ]:
tf.reduce_sum(tfa)

In [ ]:
# 生成tf.record文件
# 本身项目标数据结构
ar3=df3['data_signal'].values

In [ ]:
def parse_tf_example(example_proto):

    feature_spec = {}

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(1), dtype=tf.float32)
    
    for feature_name in CATEGORICAL_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(1), dtype=tf.string)
    
    feature_spec[TARGET_NAME] = tf.FixedLenFeature(shape=(1), dtype=tf.float32)

    parsed_features = tf.parse_example(serialized=example_proto, features=feature_spec)
    
    target = parsed_features.pop(TARGET_NAME)
    
    return parsed_features, target


def process_features(features):
    
    # example of clipping
    features['x'] = tf.clip_by_value(features['x'], clip_value_min=-3, clip_value_max=3)
    features['y'] = tf.clip_by_value(features['y'], clip_value_min=-3, clip_value_max=3)
    
    # example of polynomial expansion
    features["x_2"] = tf.square(features['x'])
    features["y_2"] = tf.square(features['y'])
    
    # example of nonlinearity
    features["xy"] = features['x'] * features['y']
    
    # example of custom logic
    features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    features["sin_x"] = tf.sin(features['x'])
    features["cos_y"] = tf.sin(features['y'])    
    return features

def tfrecods_input_fn(files_name_pattern, mode=tf.estimator.ModeKeys.EVAL, 
                 num_epochs=None, 
                 batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    
    print("")
    print("* data input_fn:")
    print("================")
    print("Input file(s): {}".format(files_name_pattern))
    print("Batch size: {}".format(batch_size))
    print("Epoch Count: {}".format(num_epochs))
    print("Mode: {}".format(mode))
    print("Shuffle: {}".format(shuffle))
    print("================")
    print("")

    file_names = tf.matching_files(files_name_pattern)
    dataset = data.TFRecordDataset(filenames=file_names)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example))
    
    if PROCESS_FEATURES:
        dataset = dataset.map(lambda features, target: (process_features(features), target))
        
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    
    features, target = iterator.get_next()
    return features, target

In [ ]:
movie_name_list = tf.train.BytesList(value=[b'The Shawshank Redemption', b'Fight Club'])
movie_rating_list = tf.train.FloatList(value=[9.0, 9.7, 5.5])

movie_names = tf.train.Feature(bytes_list=movie_name_list)
movie_ratings = tf.train.Feature(float_list=movie_rating_list)

In [ ]:
movie_dict = { 'Movie Names': movie_names, 'Movie Ratings': movie_ratings}
movies = tf.train.Features(feature=movie_dict)
example = tf.train.Example(features=movies)

In [ ]:
tf.parse_example(example)

In [ ]:
example

In [ ]:
with tf.python_io.TFRecordWriter('movie_ratings.tfrecord') as writer:
    writer.write(example.SerializeToString())

In [ ]:
psg_int

In [ ]:
df2.query("(time>737201.8932291215)and(time<737201.8935763418)")

In [ ]:
30*256

In [ ]:
dt=datetime.fromordinal(int(matlab_datenum) - 366) + timedelta(days=matlab_datenum%1)

In [ ]:
plt.figure(figsize=(100,30))
plt.plot(df['time'],[1]*len(df))
plt.plot(df2['time'],[2]*len(df2))
plt.show()

In [ ]:
df.merge(df2,on='time')

In [ ]:
dt.time()

In [ ]:
df2[df2["time"]==737201.9848952909]

In [ ]:
for i in df2.query("time>737201.9 and time<737201.99")['time']:print(i)

In [ ]:
td=parse_mat_time(df['time'][0])-parse_mat_time(df['time'][1])


In [ ]:
parse_mat_time(df['time'][0])

In [ ]:
parse_mat_time(df['time'][1])

In [ ]:
td.seconds

In [ ]:
td.total_seconds()

In [ ]:
t1=parse_mat_time(df['time'][0])
t2=parse_mat_time(df['time'][1])


In [ ]:
t1.second,t2.second